# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [3]:
import psycopg2
import pandas as pd
from scipy import stats
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import scipy
import numpy as np
pd.set_option('display.max_columns', None)  #set pandas to display all columns
import sklearn
import sklearn.linear_model

%matplotlib inline

conn = psycopg2.connect(database='mid_term_project',user='lhl_student',password='lhl_student', host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com', port='5432')

print('Connection opened successfully')

Connection opened successfully


In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [14]:
def data_preparation(data):
    delays = ['arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']
      
    try:
        dependent = data[delays]
        data = data.drop(delays, axis=1)
    except:
        pass
    
    data['week'] = pd.to_datetime(data['fl_date']).dt.week   #adds week of year column for weekly mean delay
    data['fl_date'] = pd.to_datetime(data['fl_date'])        #converts flight date column into a datetime object
    data['year'] = pd.DatetimeIndex(data['fl_date']).year    #deconstructs flight date into year
    data['month'] = pd.DatetimeIndex(data['fl_date']).month  #deconstructs flight date into month
    data['day'] = pd.DatetimeIndex(data['fl_date']).day      #deconstructs flight date into day
    data['crs_dep_hour'] = (data['crs_dep_time'] // 100)     #converts departure time to an hour
    data['crs_arr_hour'] = (data['crs_arr_time'] // 100)     #converts arrival time to an hour
    
    #Feature engineering delay by airport by week
    try:
        # Reads saved origin_mean_delay csv to dataframe
        origin_mean_delay = pd.read_csv('origin_weekly_mean_delay.csv')
    except:
        #Generates an origin_mean_delay dataframe from information on SQL server if origin_mean_delay.csv does not exist and writes it to disk for future use
        column_names = ['fl_date','arr_delay','origin']
        origin_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,origin FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        origin_delay['week'] = pd.to_datetime(origin_delay['fl_date']).dt.week
        origin_mean_delay = origin_delay.groupby(['origin', origin_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_origin_delay'})
        origin_mean_delay.to_csv('origin_weekly_mean_delay.csv',index=False)
    try:
        # Reads saved dest_mean_delay csv to dataframe
        dest_mean_delay = pd.read_csv('dest_weekly_mean_delay.csv')
    except:
        #Generates an dest_mean_delay dataframe from information on SQL server if dest_mean_delay.csv does not exist and writes it to disk for future use
        column_names = ['fl_date','arr_delay','dest']
        dest_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        dest_delay['week'] = pd.to_datetime(dest_delay['fl_date']).dt.week
        dest_mean_delay = dest_delay.groupby(['dest', dest_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_dest_delay'})
        dest_mean_delay.to_csv('dest_weekly_mean_delay.csv',index=False)
        
    #merges two mean_delay dataframes on test data based on location and week of year
    data = data.merge(dest_mean_delay, how='left', left_on=['dest','week'], right_on=['dest','week'])
    data = data.merge(origin_mean_delay, how='left', left_on=['origin','week'], right_on=['origin','week'])
    
    # Feature engineering delay by airport by hour
    try:
        #Reads saved origin_hourly_mean_delay csv to dataframe
        origin_hourly_mean_delay = pd.read_csv('origin_hourly_mean_delay.csv')
    except:
        #Generates an origin_mean_delay dataframe from information on SQL server if origin_mean_delay.csv does not exist and writes it to disk for future use
        column_names = ['crs_dep_time','origin','arr_delay']
        origin_hourly_delay = postgresql_to_dataframe(conn, 'SELECT crs_dep_time,origin,arr_delay FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        origin_hourly_delay['crs_dep_hour'] = (origin_hourly_delay['crs_dep_time'] // 100)
        origin_hourly_mean_delay = origin_hourly_delay.groupby(['origin','crs_dep_hour']).mean().reset_index().rename(columns={'arr_delay':'mean_hourly_origin_delay'})
        origin_hourly_mean_delay.to_csv('origin_hourly_mean_delay.csv', index=False)
    try:
        #Reads saved dest_hourly_mean_delay csv to dataframe
        dest_hourly_mean_delay = pd.read_csv('dest_hourly_mean_delay.csv')
    except:
        #Generates an origin_mean_delay dataframe from information on SQL server if origin_mean_delay.csv does not exist and writes it to disk for future use
        column_names = ['crs_arr_time','dest','arr_delay']
        dest_hourly_delay = postgresql_to_dataframe(conn, 'SELECT crs_arr_time,dest,arr_delay FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        dest_hourly_delay['crs_arr_hour'] = (dest_hourly_delay['crs_arr_time'] // 100)
        dest_hourly_mean_delay = dest_hourly_delay.groupby(['dest', 'crs_arr_hour']).mean().reset_index().rename(columns={'arr_delay':'mean_hourly_dest_delay'})
        dest_hourly_mean_delay.to_csv('dest_hourly_mean_delay.csv', index=False)
        
    #merges two mean_hourly_delay dataframes on test data based on location and hour
    data = data.merge(dest_hourly_mean_delay, how='left', left_on=['dest','crs_arr_hour'], right_on=['dest','crs_arr_hour'])
    data = data.merge(origin_hourly_mean_delay, how='left', left_on=['origin','crs_dep_hour'], right_on=['origin','crs_dep_hour'])    
    
    # Feature engineering traffic by airport by week
    try:
        # Reads saved dest_weekly_airport_traffic csv to dataframe
        dest_airport_traffic = pd.read_csv('dest_weekly_airport_traffic.csv')
    except:
        #Generates an dest_airport_traffic dataframe from information on SQL server if dest_weekly_airport_traffic.csv does not exist and writes it to disk for future use
        column_names = ['fl_date','arr_delay','dest']
        dest_traffic = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        dest_traffic['week'] = pd.to_datetime(dest_traffic['fl_date']).dt.week
        dest_airport_traffic = dest_traffic[['week','dest']].groupby(['dest','week']).size().reset_index(name='arriving_flights_per_week')
        dest_airport_traffic.to_csv('dest_weekly_airport_traffic.csv',index=False)
    try:
        # Reads saved origin_weekly_airport_traffic csv to dataframe
        origin_airport_traffic = pd.read_csv('origin_weekly_airport_traffic.csv')
    except:
        #Generates an origin_airport_traffic dataframe from information on SQL server if origin_weekly_airport_traffic.csv does not exist and writes it to disk for future use
        column_names = ['fl_date','arr_delay','origin']
        origin_traffic = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        origin_traffic['week'] = pd.to_datetime(origin_traffic['fl_date']).dt.week
        origin_airport_traffic = origin_traffic[['week','origin']].groupby(['origin','week']).size().reset_index(name='leaving_flights_per_week')
        origin_airport_traffic.to_csv('origin_weekly_airport_traffic.csv',index=False)
        
    #merges two weekly airport traffic dataframes on test data based on location and week
    data = data.merge(dest_airport_traffic, how='left', left_on=['dest','week'], right_on=['dest','week'])
    data = data.merge(origin_airport_traffic, how='left', left_on=['origin','week'], right_on=['origin','week'])
    
    #Feature engineering traffic by airport by hour
    try:
        # Reads saved dest_hourly_airport_traffic csv to dataframe
        dest_airport_traffic_hourly = pd.read_csv('dest_hourly_airport_traffic.csv')
    except:
        #Generates an dest_airport_traffic_hourly dataframe from information on SQL server if dest_hourly_airport_traffic.csv does not exist and writes it to disk for future use
        column_names = ['crs_arr_time','dest']
        dest_traffic = postgresql_to_dataframe(conn, 'SELECT crs_arr_time,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        dest_traffic['crs_arr_hour'] = (dest_traffic['crs_arr_time'] // 100)     #converts arrival time to an hour
        dest_airport_traffic_hourly = dest_traffic[['crs_arr_hour','dest']].groupby(['dest','crs_arr_hour']).size().reset_index(name='arriving_flights_per_hour')
        dest_airport_traffic_hourly.to_csv('dest_hourly_airport_traffic.csv',index=False)
    try:
        # Reads saved origin_hourly_airport_traffic csv to dataframe
        origin_airport_traffic_hourly = pd.read_csv('origin_hourly_airport_traffic.csv')
    except:
        #Generates an origin_airport_traffic_hourly dataframe from information on SQL server if origin_hourly_airport_traffic.csv does not exist and writes it to disk for future use
        column_names = ['crs_dep_time','origin']
        origin_traffic = postgresql_to_dataframe(conn, 'SELECT crs_arr_time,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
        origin_traffic['crs_dep_hour'] = (origin_traffic['crs_dep_time'] // 100)     #converts departure time to an hour
        origin_airport_traffic_hourly = origin_traffic[['crs_dep_hour','origin']].groupby(['origin','crs_dep_hour']).size().reset_index(name='departing_flights_per_hour')
        origin_airport_traffic_hourly.to_csv('origin_hourly_airport_traffic.csv',index=False)
    
    #merges two hourly airport traffic dataframes on test data based on location and hour
    data = data.merge(dest_airport_traffic_hourly, how='left', left_on=['dest','crs_arr_hour'], right_on=['dest','crs_arr_hour'])
    data = data.merge(origin_airport_traffic_hourly, how='left', left_on=['origin','crs_dep_hour'], right_on=['origin','crs_dep_hour'])    
    
    #adds encoding to the type of flight based on the flight number
    #mainline = 0, regional = 1, ferry = 2
    data['flight_type'] = np.select (
        [ 
         (data['mkt_carrier_fl_num'].between(1, 2949, inclusive=True)),
         (data['mkt_carrier_fl_num'].between(2950, 7999, inclusive=True)),
         (data['mkt_carrier_fl_num'].between(8000, 9400, inclusive=True)),
        ], 
        [
         'mainline',
         'regional',
         'ferry'
        ]
                                        )
    data = pd.concat([data,pd.get_dummies(data['flight_type'], prefix = 'flight_type')], axis = 1)
    
    #adds encoding to the flight length based on the distance of the flight
    #short = 0, medium = 1, long = 2
    data['flight_length'] = np.select (
        [ 
         (data['distance'] < 800),
         (data['distance'].between(800, 2200, inclusive=False)),
         (data['distance'] >=2200)
        ], 
        [
         'short',
         'medium',
         'long'
        ]
                                        )
    data = pd.concat([data,pd.get_dummies(data['flight_length'], prefix = 'flight_haul')], axis = 1)
    
    #adds encoding for carriers to differentiate different carriers
    data = pd.concat([data,pd.get_dummies(data['mkt_unique_carrier'], prefix = 'carrier')], axis = 1)
    
    final_df = data[['year','month','week','day','crs_dep_hour','crs_arr_hour','mkt_carrier_fl_num','mean_weekly_dest_delay','mean_weekly_origin_delay','mean_hourly_origin_delay','mean_hourly_dest_delay',
                     'arriving_flights_per_week','leaving_flights_per_week','arriving_flights_per_hour','departing_flights_per_hour','flight_type_0','flight_type_ferry','flight_type_mainline',
                     'flight_type_regional','flight_haul_long','flight_haul_medium','flight_haul_short','carrier_AA','carrier_B6','carrier_DL','carrier_F9','carrier_G4','carrier_HA','carrier_NK','carrier_UA',
                     'carrier_VX','carrier_WN']]
    if len(dependent) >= 1:
        final_df = pd.concat([final_df, dependent], axis=1)
    
    return final_df
    

In [9]:
column_names = ['fl_date','mkt_unique_carrier','mkt_carrier_fl_num','origin','dest','crs_dep_time','crs_arr_time','crs_elapsed_time','distance','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']
data = postgresql_to_dataframe(conn, 'SELECT fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay FROM flights WHERE cancelled = 0 and diverted = 0', column_names)


In [22]:
data = data.dropna(subset=['arr_delay','crs_elapsed_time'])

In [24]:
data = data.fillna(0)

In [25]:
data.isna().sum(axis=0)

fl_date                0
mkt_unique_carrier     0
mkt_carrier_fl_num     0
origin                 0
dest                   0
crs_dep_time           0
crs_arr_time           0
crs_elapsed_time       0
distance               0
arr_delay              0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [27]:
data.describe()

,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07,1.561574e+07
mean,2.725578e+03,1.328541e+03,1.483496e+03,1.381119e+02,7.703655e+02,5.475922e+00,3.926706e+00,7.081429e-01,3.129184e+00,1.706214e-02,5.259770e+00
std,1.861612e+03,4.896341e+02,5.165631e+02,7.181271e+01,5.872203e+02,4.987399e+01,2.888873e+01,1.373737e+01,1.776831e+01,1.377091e+00,2.577453e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,-1.430000e+02,1.600000e+01,-1.290000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.119000e+03,9.150000e+02,1.100000e+03,8.700000e+01,3.420000e+02,-1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.305000e+03,1.320000e+03,1.510000e+03,1.190000e+02,6.050000e+02,-6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.307000e+03,1.735000e+03,1.916000e+03,1.680000e+02,1.005000e+03,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.401000e+03,2.359000e+03,2.400000e+03,1.645000e+03,5.095000e+03,2.973000e+03,2.973000e+03,2.900000e+03,1.848000e+03,1.078000e+03,2.962000e+03


In [29]:
data_nooutliers = data[(data['arr_delay'] < 250)&(data['arr_delay'] > -60)].copy(deep=True)

In [30]:
data = data_nooutliers.sample(5000000)

In [31]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
11716356,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0
13860151,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0
8492833,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0
13160591,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0
4489885,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542987,2019-10-03,AA,35,DFW,SAT,1221,1329,68.0,247.0,-17.0,0.0,0.0,0.0,0.0,0.0
7171879,2019-06-17,DL,3500,IAD,DTW,1321,1459,98.0,383.0,-19.0,0.0,0.0,0.0,0.0,0.0
9312124,2019-02-24,UA,743,ORD,SFO,1816,2054,278.0,1846.0,22.0,0.0,0.0,22.0,0.0,0.0
5692182,2019-10-09,DL,1965,DEN,LGA,1120,1710,230.0,1620.0,74.0,0.0,0.0,74.0,0.0,0.0


In [32]:
data.to_csv('training_data.csv',index=False)

In [52]:
data = pd.read_csv('training_data.csv')

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [33]:
data['week'] = pd.to_datetime(data['fl_date']).dt.week

In [34]:
data['fl_date'] = pd.to_datetime(data['fl_date'])

In [35]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week
11716356,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42
13860151,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4
8492833,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33
13160591,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39
4489885,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542987,2019-10-03,AA,35,DFW,SAT,1221,1329,68.0,247.0,-17.0,0.0,0.0,0.0,0.0,0.0,40
7171879,2019-06-17,DL,3500,IAD,DTW,1321,1459,98.0,383.0,-19.0,0.0,0.0,0.0,0.0,0.0,25
9312124,2019-02-24,UA,743,ORD,SFO,1816,2054,278.0,1846.0,22.0,0.0,0.0,22.0,0.0,0.0,8
5692182,2019-10-09,DL,1965,DEN,LGA,1120,1710,230.0,1620.0,74.0,0.0,0.0,74.0,0.0,0.0,41


#### Origin Mean Delay Per Week

In [62]:
#Generates an origin_mean_delay dataframe from information on SQL server

column_names = ['fl_date','arr_delay','origin']
origin_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,origin FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
origin_delay['week'] = pd.to_datetime(origin_delay['fl_date']).dt.week
origin_mean_delay = origin_delay.groupby(['origin', origin_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_origin_delay'})
origin_mean_delay.to_csv('origin_weekly_mean_delay.csv',index=False)
origin_mean_delay

,origin,week,mean_weekly_origin_delay
0,ABE,1,12.616915
1,ABE,2,8.386740
2,ABE,3,9.852941
3,ABE,4,3.483146
4,ABE,5,13.156069
...,...,...,...
19077,YUM,48,-7.142857
19078,YUM,49,-6.371429
19079,YUM,50,11.333333
19080,YUM,51,18.750000


In [36]:
# Reads saved origin_mean_delay csv to dataframe
origin_mean_delay = pd.read_csv('origin_weekly_mean_delay.csv')

#### Destination Mean Delay Per Week

In [37]:
# Reads saved dest_mean_delay csv to dataframe
dest_mean_delay = pd.read_csv('dest_weekly_mean_delay.csv')

In [63]:
#Generates an dest_mean_delay dataframe from information on SQL server

column_names = ['fl_date','arr_delay','dest']
dest_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
dest_delay['week'] = pd.to_datetime(dest_delay['fl_date']).dt.week
dest_mean_delay = dest_delay.groupby(['dest', dest_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_dest_delay'})
dest_mean_delay.to_csv('dest_weekly_mean_delay.csv',index=False)
dest_mean_delay

,dest,week,mean_weekly_dest_delay
0,ABE,1,4.920000
1,ABE,2,8.855556
2,ABE,3,8.362573
3,ABE,4,4.302198
4,ABE,5,3.350877
...,...,...,...
19078,YUM,48,1.095238
19079,YUM,49,1.300000
19080,YUM,50,3.876712
19081,YUM,51,11.736842


In [38]:
data = data.merge(dest_mean_delay, how='left', left_on=['dest','week'], right_on=['dest','week'])
data = data.merge(origin_mean_delay, how='left', left_on=['origin','week'], right_on=['origin','week'])

In [39]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2019-10-03,AA,35,DFW,SAT,1221,1329,68.0,247.0,-17.0,0.0,0.0,0.0,0.0,0.0,40,-3.606776,5.486009
4999996,2019-06-17,DL,3500,IAD,DTW,1321,1459,98.0,383.0,-19.0,0.0,0.0,0.0,0.0,0.0,25,7.724709,20.777567
4999997,2019-02-24,UA,743,ORD,SFO,1816,2054,278.0,1846.0,22.0,0.0,0.0,22.0,0.0,0.0,8,3.700796,15.551226
4999998,2019-10-09,DL,1965,DEN,LGA,1120,1710,230.0,1620.0,74.0,0.0,0.0,74.0,0.0,0.0,41,24.161236,12.791009


#### Airport Traffic

In [68]:
dest_airport_traffic = dest_delay[['week','dest']].groupby(['dest','week']).size().reset_index(name='arriving_flights_per_week')

In [40]:
# Reads saved dest_weekly_airport_traffic csv to dataframe
dest_airport_traffic = pd.read_csv('dest_weekly_airport_traffic.csv')

In [72]:
dest_airport_traffic.to_csv('dest_weekly_airport_traffic.csv',index=False)

In [69]:
origin_airport_traffic = origin_delay[['week','origin']].groupby(['origin','week']).size().reset_index(name='leaving_flights_per_week')

In [41]:
# Reads saved origin_weekly_airport_traffic csv to dataframe
origin_airport_traffic = pd.read_csv('origin_weekly_airport_traffic.csv')

In [73]:
origin_airport_traffic.to_csv('origin_weekly_airport_traffic.csv',index=False)

In [42]:
data = data.merge(dest_airport_traffic, how='left', left_on=['dest','week'], right_on=['dest','week'])
data = data.merge(origin_airport_traffic, how='left', left_on=['origin','week'], right_on=['origin','week'])

In [43]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2019-10-03,AA,35,DFW,SAT,1221,1329,68.0,247.0,-17.0,0.0,0.0,0.0,0.0,0.0,40,-3.606776,5.486009,1653,11329
4999996,2019-06-17,DL,3500,IAD,DTW,1321,1459,98.0,383.0,-19.0,0.0,0.0,0.0,0.0,0.0,25,7.724709,20.777567,7135,3682
4999997,2019-02-24,UA,743,ORD,SFO,1816,2054,278.0,1846.0,22.0,0.0,0.0,22.0,0.0,0.0,8,3.700796,15.551226,6534,13704
4999998,2019-10-09,DL,1965,DEN,LGA,1120,1710,230.0,1620.0,74.0,0.0,0.0,74.0,0.0,0.0,41,24.161236,12.791009,6537,10967


#### Airport Encoding

In [44]:
#mainline = 0, regional = 1, ferry = 2
data['flight_type'] = np.select (
    [ 
     (data['mkt_carrier_fl_num'] .between(1, 2949, inclusive=True)),
     (data['mkt_carrier_fl_num'].between(2950, 7999, inclusive=True)),
     (data['mkt_carrier_fl_num'].between(8000, 9400, inclusive=True)),
    ], 
    [
     'mainline',
     'regional',
     'ferry'
    ]
                                    )
data = pd.concat([data,pd.get_dummies(data['flight_type'], prefix = 'flight_type')], axis = 1)
print(data.shape)
data.head()

(5000000, 25)


,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,regional,0,0,0,1
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,mainline,0,0,1,0
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,mainline,0,0,1,0
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,mainline,0,0,1,0
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,mainline,0,0,1,0


#### Flight Haul Lengths

In [45]:
#short = 0, medium = 1, long = 2
data['flight_length'] = np.select (
    [ 
     (data['distance'] < 800),
     (data['distance'].between(800, 2200, inclusive=False)),
     (data['distance'] >=2200)
    ], 
    [
     'short',
     'medium',
     'long'
    ]
                                    )
data = pd.concat([data,pd.get_dummies(data['flight_length'], prefix = 'flight_haul')], axis = 1)
print(data.shape)
data.head()

(5000000, 29)


,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,regional,0,0,0,1,short,0,0,1
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,mainline,0,0,1,0,short,0,0,1
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,mainline,0,0,1,0,long,1,0,0
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,mainline,0,0,1,0,short,0,0,1
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,mainline,0,0,1,0,short,0,0,1


#### Adding Hourly Arrival and Departure Times

In [46]:
data['crs_dep_hour'] = (data['crs_dep_time'] // 100)
data['crs_arr_hour'] = (data['crs_arr_time'] // 100)

#### Breaking fl_date into year, month day

In [47]:
data['year'] = pd.DatetimeIndex(data['fl_date']).year
data['month'] = pd.DatetimeIndex(data['fl_date']).month
data['day'] = pd.DatetimeIndex(data['fl_date']).day

In [48]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,year,month,day
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,regional,0,0,0,1,short,0,0,1,13,14,2018,10,17
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,mainline,0,0,1,0,short,0,0,1,18,19,2019,1,27
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,mainline,0,0,1,0,long,1,0,0,18,21,2019,8,16
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,mainline,0,0,1,0,short,0,0,1,9,11,2019,9,23
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,mainline,0,0,1,0,short,0,0,1,13,14,2018,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2019-10-03,AA,35,DFW,SAT,1221,1329,68.0,247.0,-17.0,0.0,0.0,0.0,0.0,0.0,40,-3.606776,5.486009,1653,11329,mainline,0,0,1,0,short,0,0,1,12,13,2019,10,3
4999996,2019-06-17,DL,3500,IAD,DTW,1321,1459,98.0,383.0,-19.0,0.0,0.0,0.0,0.0,0.0,25,7.724709,20.777567,7135,3682,regional,0,0,0,1,short,0,0,1,13,14,2019,6,17
4999997,2019-02-24,UA,743,ORD,SFO,1816,2054,278.0,1846.0,22.0,0.0,0.0,22.0,0.0,0.0,8,3.700796,15.551226,6534,13704,mainline,0,0,1,0,medium,0,1,0,18,20,2019,2,24
4999998,2019-10-09,DL,1965,DEN,LGA,1120,1710,230.0,1620.0,74.0,0.0,0.0,74.0,0.0,0.0,41,24.161236,12.791009,6537,10967,mainline,0,0,1,0,medium,0,1,0,11,17,2019,10,9


In [49]:
data = pd.concat([data,pd.get_dummies(data['mkt_unique_carrier'], prefix = 'carrier')], axis = 1)
print(data.shape)
data.head()

(5000000, 45)


,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,year,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,2018-10-17,DL,4917,MDW,MSP,1303,1430,87.0,349.0,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,regional,0,0,0,1,short,0,0,1,13,14,2018,10,17,0,0,0,1,0,0,0,0,0,0,0
1,2019-01-27,UA,681,SFO,LAS,1822,1948,86.0,414.0,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,mainline,0,0,1,0,short,0,0,1,18,19,2019,1,27,0,0,0,0,0,0,0,0,1,0,0
2,2019-08-16,UA,1075,EWR,SNA,1830,2128,358.0,2434.0,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,mainline,0,0,1,0,long,1,0,0,18,21,2019,8,16,0,0,0,0,0,0,0,0,1,0,0
3,2019-09-23,WN,826,LAX,SJC,945,1100,75.0,308.0,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,mainline,0,0,1,0,short,0,0,1,9,11,2019,9,23,0,0,0,0,0,0,0,0,0,0,1
4,2018-01-15,AS,2273,SEA,RDM,1335,1436,61.0,228.0,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,mainline,0,0,1,0,short,0,0,1,13,14,2018,1,15,0,1,0,0,0,0,0,0,0,0,0


In [50]:
feat_engineered_training_data = data.drop(['fl_date','mkt_unique_carrier','origin','dest','crs_dep_time','crs_arr_time','distance','flight_type','flight_length','year','crs_elapsed_time'],axis=1)
feat_engineered_training_data

,mkt_carrier_fl_num,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,4917,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,0,0,0,1,0,0,1,13,14,10,17,0,0,0,1,0,0,0,0,0,0,0
1,681,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,0,0,1,0,0,0,1,18,19,1,27,0,0,0,0,0,0,0,0,1,0,0
2,1075,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,0,0,1,0,1,0,0,18,21,8,16,0,0,0,0,0,0,0,0,1,0,0
3,826,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,0,0,1,0,0,0,1,9,11,9,23,0,0,0,0,0,0,0,0,0,0,1
4,2273,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,0,0,1,0,0,0,1,13,14,1,15,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,35,-17.0,0.0,0.0,0.0,0.0,0.0,40,-3.606776,5.486009,1653,11329,0,0,1,0,0,0,1,12,13,10,3,1,0,0,0,0,0,0,0,0,0,0
4999996,3500,-19.0,0.0,0.0,0.0,0.0,0.0,25,7.724709,20.777567,7135,3682,0,0,0,1,0,0,1,13,14,6,17,0,0,0,1,0,0,0,0,0,0,0
4999997,743,22.0,0.0,0.0,22.0,0.0,0.0,8,3.700796,15.551226,6534,13704,0,0,1,0,0,1,0,18,20,2,24,0,0,0,0,0,0,0,0,1,0,0
4999998,1965,74.0,0.0,0.0,74.0,0.0,0.0,41,24.161236,12.791009,6537,10967,0,0,1,0,0,1,0,11,17,10,9,0,0,0,1,0,0,0,0,0,0,0


In [51]:
feat_engineered_training_data.to_csv('Feat_engineered_training_data.csv', index=False)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [3]:
training_df = pd.read_csv('Feat_engineered_training_data.csv')

In [4]:
training_df

,mkt_carrier_fl_num,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,4917,-6.0,0.0,0.0,0.0,0.0,0.0,42,-3.669272,1.080318,6398,3399,0,0,0,1,0,0,1,13,14,10,17,0,0,0,1,0,0,0,0,0,0,0
1,681,-5.0,0.0,0.0,0.0,0.0,0.0,4,-3.190229,-0.628954,5977,6355,0,0,1,0,0,0,1,18,19,1,27,0,0,0,0,0,0,0,0,1,0,0
2,1075,29.0,0.0,0.0,29.0,0.0,0.0,33,1.425821,33.765468,1766,5868,0,0,1,0,1,0,0,18,21,8,16,0,0,0,0,0,0,0,0,1,0,0
3,826,9.0,0.0,0.0,0.0,0.0,0.0,39,-2.012551,-1.997789,2709,9498,0,0,1,0,0,0,1,9,11,9,23,0,0,0,0,0,0,0,0,0,0,1
4,2273,-13.0,0.0,0.0,0.0,0.0,0.0,3,11.181481,-0.681825,270,6377,0,0,1,0,0,0,1,13,14,1,15,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,35,-17.0,0.0,0.0,0.0,0.0,0.0,40,-3.606776,5.486009,1653,11329,0,0,1,0,0,0,1,12,13,10,3,1,0,0,0,0,0,0,0,0,0,0
4999996,3500,-19.0,0.0,0.0,0.0,0.0,0.0,25,7.724709,20.777567,7135,3682,0,0,0,1,0,0,1,13,14,6,17,0,0,0,1,0,0,0,0,0,0,0
4999997,743,22.0,0.0,0.0,22.0,0.0,0.0,8,3.700796,15.551226,6534,13704,0,0,1,0,0,1,0,18,20,2,24,0,0,0,0,0,0,0,0,1,0,0
4999998,1965,74.0,0.0,0.0,74.0,0.0,0.0,41,24.161236,12.791009,6537,10967,0,0,1,0,0,1,0,11,17,10,9,0,0,0,1,0,0,0,0,0,0,0


In [5]:
delays = ['arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']

In [6]:
X = training_df.drop(delays, axis=1)
y = training_df['arr_delay']

In [7]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size=0.2)

In [8]:
lr = sklearn.linear_model.LinearRegression()

In [9]:
lr.fit(X_train,y_train)

LinearRegression()

In [10]:
y_pred = lr.predict(X_test)

In [11]:
R2 = sklearn.metrics.r2_score(y_test,y_pred)

In [12]:
R2

0.051464902579866356

In [14]:
X_train

,mkt_carrier_fl_num,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
2684871,3198,27,11.775203,7.118164,5179,1025,0,0,0,1,0,1,0,12,14,7,5,0,0,0,0,0,0,0,0,0,0,1
1390795,3456,9,5.329337,2.935698,2669,3468,0,0,0,1,0,1,0,9,15,3,2,0,1,0,0,0,0,0,0,0,0,0
520743,671,16,3.084881,4.811177,378,15126,0,0,1,0,0,0,1,21,22,4,20,0,0,0,1,0,0,0,0,0,0,0
4798024,1381,18,9.930602,8.365348,14498,9454,0,0,1,0,0,0,1,16,17,4,30,1,0,0,0,0,0,0,0,0,0,0
1423945,1702,44,0.001090,-1.781804,6424,6485,0,0,1,0,0,0,1,8,10,10,30,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121197,4544,33,7.101848,14.421415,2221,5370,0,0,0,1,0,1,0,20,21,8,14,1,0,0,0,0,0,0,0,0,0,0
4042470,1927,48,1.199645,5.835693,3942,5581,0,0,1,0,0,0,1,12,14,11,25,0,0,1,0,0,0,0,0,0,0,0
3356069,5003,46,6.926970,15.247312,15117,465,0,0,0,1,0,0,1,19,21,11,18,0,0,0,1,0,0,0,0,0,0,0
903114,5600,24,10.744138,6.794502,5374,6730,0,0,0,1,0,1,0,11,18,6,16,0,0,0,0,0,0,0,0,0,0,1


In [13]:
lr.coef_

array([ 5.96405590e-04,  2.86627828e-02,  5.40171946e-01,  5.10483742e-01,
        1.44718106e-04,  1.18601069e-04, -1.69685144e+01,  1.37185863e+01,
        2.10379962e+00,  1.14612847e+00, -1.71468915e+00,  4.01168414e-01,
        1.31352074e+00,  6.31828260e-01,  3.17432729e-01, -9.42280142e-02,
       -1.75864396e-02, -1.62170303e+00, -6.89373043e-01,  3.52019495e+00,
       -4.36220675e+00,  4.25012288e+00,  3.47137697e-01,  3.80355304e+00,
       -1.65845957e+00, -1.73040353e+00, -1.55156958e+00, -3.07293064e-01])

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [9]:
data = pd.read_csv('training_data.csv')

In [10]:
delays = ['arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']

In [15]:
training_data = data_preparation(data)

In [19]:
training_data.to_csv('feat_engineered_v2.csv',index=False)

In [16]:
training_data

,year,month,week,day,crs_dep_hour,crs_arr_hour,mkt_carrier_fl_num,mean_weekly_dest_delay,mean_weekly_origin_delay,mean_hourly_origin_delay,mean_hourly_dest_delay,arriving_flights_per_week,leaving_flights_per_week,arriving_flights_per_hour,departing_flights_per_hour,flight_type_0,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,carrier_AA,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018,10,42,17,13,14,4917,-3.669272,1.080318,5.610564,0.095364,6398,3399,23073,11824.0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,-6.0,0.0,0.0,0.0,0.0,0.0
1,2019,1,4,27,18,19,681,-3.190229,-0.628954,7.694459,8.998472,5977,6355,19628,17553.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-5.0,0.0,0.0,0.0,0.0,0.0
2,2019,8,33,16,18,21,1075,1.425821,33.765468,26.583710,3.382393,1766,5868,7372,15016.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,29.0,0.0,0.0,29.0,0.0,0.0
3,2019,9,39,23,9,11,826,-2.012551,-1.997789,0.534735,-0.556910,2709,9498,9023,29063.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,9.0,0.0,0.0,0.0,0.0,0.0
4,2018,1,3,15,13,14,2273,11.181481,-0.681825,3.862733,6.074826,270,6377,1577,10718.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-13.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2019,10,40,3,12,13,35,-3.606776,5.486009,9.074566,0.406943,1653,11329,5588,20316.0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-17.0,0.0,0.0,0.0,0.0,0.0
4999996,2019,6,25,17,13,14,3500,7.724709,20.777567,2.475063,0.452854,7135,3682,36688,1814.0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,-19.0,0.0,0.0,0.0,0.0,0.0
4999997,2019,2,8,24,18,20,743,3.700796,15.551226,19.694393,17.835139,6534,13704,22977,55180.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,22.0,0.0,0.0,22.0,0.0,0.0
4999998,2019,10,41,9,11,17,1965,24.161236,12.791009,5.062005,17.924412,6537,10967,22322,25529.0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,74.0,0.0,0.0,74.0,0.0,0.0


In [17]:
import xgboost as xgb

In [20]:
X = training_data.drop(delays, axis=1)
y = training_data['arr_delay']

In [21]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 5, n_estimators = 250)

In [24]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=15,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=250, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=5, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
y_pred = xg_reg.predict(X_test)

In [26]:
R2 = sklearn.metrics.r2_score(y_test,y_pred)

In [27]:
R2

0.14901874657509762

In [28]:
from sklearn.metrics import mean_squared_error

In [29]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 31.821750


In [31]:
import pickle

In [32]:
filename = 'R2-0149-XGB.pickle'
pickle.dump(xg_reg, open(filename, 'wb'))

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

In [ ]:
import xgboost as xgb
import pickle
import pandas as pd
import numpy as np

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.